### Q1>
### Write a python program to implement Logistic Regression for multiclass classification from scratch
### using the following dataset.
### Link: https://www.kaggle.com/uciml/iris

#### Dependencies and Libraries import

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#### Import Dataset

In [120]:
data = pd.read_csv('Iris.csv',index_col='Id')
data = data.sample(frac=1).reset_index(drop = True)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.5,2.6,4.4,1.2,Iris-versicolor
1,5.8,2.8,5.1,2.4,Iris-virginica
2,6.5,3.0,5.8,2.2,Iris-virginica
3,6.0,3.0,4.8,1.8,Iris-virginica
4,5.1,3.8,1.6,0.2,Iris-setosa


In [121]:
m = data.shape[0]
print("The number of rows in this dataset is : ",m)

The number of rows in this dataset is :  150


In [122]:
n = data.shape[1]
print("The number of features in this dataset is : ",n-1)

The number of features in this dataset is :  4


#### Categorizing Data

In [123]:
target = pd.get_dummies(data['Species'])
target.shape

(150, 3)

In [124]:
target

,Iris-setosa,Iris-versicolor,Iris-virginica
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,1,0,0
...,...,...,...
145,0,1,0
146,0,0,1
147,1,0,0
148,0,1,0


In [125]:
data.drop(labels = "Species", axis =1, inplace =True)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.5,2.6,4.4,1.2
1,5.8,2.8,5.1,2.4
2,6.5,3.0,5.8,2.2
3,6.0,3.0,4.8,1.8
4,5.1,3.8,1.6,0.2


#### Normalizing Data

In [126]:
scaler = StandardScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,-0.416010,-1.050569,0.364699,0.001753
1,-0.052506,-0.587764,0.762759,1.579429
2,0.795669,-0.124958,1.160819,1.316483
3,0.189830,-0.124958,0.592162,0.790591
4,-0.900681,1.726266,-1.227541,-1.312977
...,...,...,...,...
145,0.068662,-0.124958,0.250967,0.396172
146,-0.294842,-0.587764,0.649027,1.053537
147,-1.870024,-0.124958,-1.511870,-1.444450
148,0.916837,-0.356361,0.478430,0.133226


In [127]:
X_train, X_test, Y_train, Y_test = train_test_split(data,target,test_size=0.2)

In [128]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()
print("The shape of the X_train : {}".format(X_train.shape))
print("The shape of the Y_train : {}".format(Y_train.shape))
print("The shape of the X_test : {}".format(X_test.shape))
print("The shape of the Y_test : {}".format(Y_test.shape))

The shape of the X_train : (120, 4)
The shape of the Y_train : (120, 3)
The shape of the X_test : (30, 4)
The shape of the Y_test : (30, 3)


#### Initialize parameters

In [129]:
nc=3  ###### this represent numbers of classes here
theta=np.zeros((nc,n-1)) ##### weight parameters for classes
print(theta[0].shape)
bias = np.zeros(nc) ###### bias for classes
print(bias.shape)

(4,)
(3,)


In [148]:
def gradientdescent(x,y,w,b):
    epochs = 10000  ## we run it for 2000 times
    learning_rate = 0.0002  ## we set the learning_rate at 0.02

    ## so in every iteration of inner loop we update the weights for that particular class
    ## considering it to be binary classification with respect to other classes

    for i in range(epochs):
        for j in range(nc):
            z = b[j]+np.dot(x,w[j])
            temp1 = 1/(1+np.exp(-z))  ## this is basically the sigmoid function
            temp2 = np.array(y[:int(0.8*m),j])
            loss = temp1-temp2
            weight_grad = np.dot(x.T,loss)/m  ## we calculate the weight gradient and bias gradient
            bias_grad = np.sum(loss)/m
            w[j] = w[j]-weight_grad*learning_rate  ## we find weights for all 3 classes
            b[j] = b[j]-bias_grad*learning_rate    ## we find bias for all 3 classes
    return w,b
print(X_train.shape)
print(theta[0].shape)
theta,bias=gradientdescent(X_train,Y_train,theta,bias)

(120, 4)
(4,)


In [143]:
def findmaxprob(arr):
    m=arr[0]
    index=0
    for i in range(len(arr)):
        if(arr[i]>m):
            m=arr[i]
            index=i
    return index

In [144]:
def predict(x,w,b):
    res=[]
    for j in range(len(x)):
        pc=[]  ## this array stores probability of that particular row for all classes
        for i in range(nc):  ## we find its probability in each class
            z=b[i]+np.dot(x[j],w[i]) 
            prob=1/(1+np.exp(-z))
            pc.append(prob)
        index=findmaxprob(pc)  ## the index which has max probability will be allotted the class
        res.append(index)
    res = np.array(res)
    return np.eye(np.max(res) + 1)[res]

In [145]:
def findaccuracy(actual_res,calc_res):
    l=len(actual_res)
    acc=0
    for i in range(len(actual_res)):
        if(calc_res[i].all()==actual_res[i].all()): #where the prediction matches the true class
            acc=acc+1
    return float(acc/l*100.0)

In [146]:
res=predict(X_test,theta,bias)
print("The weights for measuring class1 against all is : ",theta[0])
print("The weights for measuring class2 against all is : ",theta[1])
print("The weights for measuring class3 against all is : ",theta[2])
print("The resultant array is : ",res)
print("The actual result is : ",Y_test)
accuracy=findaccuracy(Y_test,res)
print("The accuracy is : {} %".format(accuracy))

The weights for measuring class1 against all is :  [-0.46164017  0.47941622 -0.65336565 -0.61667244]
The weights for measuring class2 against all is :  [-0.00663819 -0.47425656  0.11859755  0.02446715]
The weights for measuring class3 against all is :  [0.43383372 0.01623339 0.50051021 0.57182425]
The resultant array is :  [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
The actual result is :  [[1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]]
The accuracy is : 1